In [3]:
import os
import pandas as pd
from dotenv import load_dotenv
import openai
import psycopg2
import time

In [4]:
load_dotenv("R:\\Richard\\src\\GitHub\\BCJTI\\translate-smart-gpt3\\.env")
openai.api_key=os.environ.get("OPENAI_API_KEY")

In [5]:
conn = psycopg2.connect(
    host=os.environ.get("PGHOST"),
    port=os.environ.get("PGPORT"),
    user=os.environ.get("PGUSER"),
    password=os.environ.get("PGPASSWORD"),
    database=os.environ.get("PGDATABASE"),
)

In [30]:
def translate(key, enUS, ptBR):
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are a translation engine that, in the contex of system which manages process of shipment of goods - comex, givin the key and the description in english and brasilian portuguese, can only translate to spanish and cannot interpret it. Return just the translation with no explanation nor comments. The key has the contex in which the message will be used, take it into account."},
        {"role": "user", "content": f"For the following key: {key}\nen-US: {enUS}\npt-BR: {ptBR}\nes-ES: "},
      ],
      temperature=0,
      n=1,
    )
    translation = response.choices[0].message.content.strip()
    return translation

translate("Caption.tpPPCCMaster", "PP / CC (Master)", "PP / CC (Master)")

'PP/CC (Maestro)'

In [7]:
def load_data():
    if os.path.exists("R:\\Richard\\src\\GitHub\\BCJTI\\translate-smart-gpt3\\translations.csv"):
        data = pd.read_csv("R:\\Richard\\src\\GitHub\\BCJTI\\translate-smart-gpt3\\translations.csv")
    else:
        with conn.cursor() as cur:
            cur.execute("SELECT dsmsgkey, dsenus, dsptbr, dseses FROM smart.i18n where dtdeleted is null")
            data = pd.DataFrame(cur.fetchall(), columns=["dsmsgkey", "dsenus", "dsptbr", "dseses"])
    return data

data = load_data()
data

,dsmsgkey,dsenus,dsptbr,dseses
0,FormatSettings.DecimalSeparator,.,",",","
1,ActionGroupTypes.AIR,Air,Aéreo,Aéreo
2,Action.name.DELETEPRICINGQUOTATIONVOLUME,Batch Delete Volumes (Pricing and Quotation),Remover Volumes em Lote (do Pricing e da Cotação),Eliminar Volumenes en Lote (de Precios y Cotiz...
3,Action.name.DELETEPRICINGVOLUME,Delete Pricing Volume,Remover Volume do Pricing,Eliminar Volumen de Precios
4,Label.nmBundle,Plan,Plano,Paquete
...,...,...,...,...
4970,rBuilder.i18nLbNF,NF,NF,NF
4971,rBuilder.i18nLbPPCC,PP/CC,PP/CC,PP/CC
4972,rBuilder.i18nLbQuantityTon,Qtd TON,Qtd TON,Cant. TON
4973,Shortcut.Accesskey.Yes,Y,S,S


In [8]:
def save_csv(data):
    data.to_csv("R:\\Richard\\src\\GitHub\\BCJTI\\translate-smart-gpt3\\translations.csv", index=False)

In [9]:
def save_data(data):
    save_csv(data)
    with conn.cursor() as cur:
        for _, row in data.iterrows():
            cur.execute(
                "UPDATE smart.i18n SET dseses = %s WHERE dsmsgkey = %s",
                (row["dseses"], row["dsmsgkey"]),
            )
        conn.commit()

In [ ]:
save_csv(data)

In [ ]:
translate("Label.Air", "Air", "Aéreo")

In [ ]:
# seta as traduções pra nulo
data["dseses"] = None

In [ ]:
for idx, row in data.iterrows():
    if pd.isna(row["dseses"]):
        try:
            translation = translate(row["dsmsgkey"], row["dsenus"], row["dsptbr"])
            data.loc[idx, "dseses"] = translation
            save_csv(data)
            print(f"Translated {row['dsmsgkey']}")
            time.sleep(3)
        except:
            print(f"Error translating {row['dsmsgkey']}")
            time.sleep(3)


In [ ]:
data["dseses"].isnull().sum()

In [ ]:
data

In [ ]:
save_csv(data)

In [10]:
save_data(data)

In [ ]:
conn.close()


In [23]:
# definindo para None a coluna dseses para registros onde dsptbr contem 'House' e dseses não contém 'House'
data_house = data.loc[(data['dsptbr'].str.contains('Master')) & (~data['dseses'].str.contains('Master'))]
data_house


,dsmsgkey,dsenus,dsptbr,dseses
39,Caption.tpPPCCMaster,PP / CC (Master),PP / CC (Master),PP/CC (Maestro)
47,eBuilder.cdCustomerReferenceMaster,Customer Ref (Master),Ref do Cliente (Master),Ref. del cliente (Maestro)
52,eBuilder.cdProcessMaster,Master,Master,Maestro
58,email.followup.lbBlMaster,BL/AWB Master,BL/AWB Master,BL/AWB Maestro
1011,Caption.cdCustomerReferenceMaster,Customer Ref (Master),Ref do Cliente (Master),Ref. del cliente (Maestro)
1015,Caption.cdFreightTypeMaster,Freight Type (Master),Tipo do Frete (Master),Tipo de flete (Maestro)
1017,Caption.cdIncotermMaster,Incoterm (Master),Incoterm (Master),Incoterm (Maestro)
1032,Caption.dsAwbMaster,AWB Master,AWB Master,AWB Maestro
1053,Caption.dsShipperMaster,Shipper(Master),Shipper(Master),Remitente (Maestro)
1074,Caption.dtProcessMaster,Process date (Master),Data do processo (Master),Fecha de proceso (Maestro)


In [ ]:
lastData = pd.read_csv("R:\\Richard\\src\\GitHub\\BCJTI\\translate-smart-gpt3\\_4_translations.csv")
lastData